In [ ]:
"""common sense QA DATA"""

In [9]:
import pandas as pd

split = 'dev'
data_cose = pd.read_json(f'./cose/{split}_cose.jsonl', lines=True)
data_csqa = pd.read_json(f'./csqa/{split}_csqa.jsonl', lines=True)

In [13]:
import random
def qa2text(question, answer):
    question = question.lower()
    question = question.replace('?', '.')
    question = question.replace('what', answer)
    question = question.replace('how', answer)
    question = question.replace('why', answer)
    question = question.replace('when', answer)
    question = question.replace('where', answer)
    return question

data_pos = []
data_neg = []
for idx, row in data_csqa.iterrows():
    str2int = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    # find the correct answer
    answerKey = row['answerKey']        # 'A' or 'B' or 'C' ...
    question = row['question']['stem']  # a statement ending with '?' containing 'what', 'how'...
    answer_t = row['question']['choices'][str2int[answerKey]]['text']
    statement_t = qa2text(question, answer_t)

    # find the incorrect answer
    str2int.pop(answerKey)
    answerKey = random.choice([k for k in str2int.keys()])
    answer_f = row['question']['choices'][str2int[answerKey]]['text']
    statement_f = qa2text(question, answer_f)

    data_pos.append({'id': row['id'],
                     'statement': statement_t,
                     'label': 1})
    data_neg.append({'id': row['id'],
                     'statement': statement_f,
                     'label': 0})


In [14]:
data_pos_e = []
data_neg_e = []
for idx, row in data_cose.iterrows():
    for sample in data_pos:
        if row['id'] == sample['id']:
            sample.update({'explanation': row['explanation']['open-ended']})
            data_pos_e.append(sample)

In [15]:
data_pos_e = pd.DataFrame(data_pos_e)
data_pos_e.to_json(f'./com2sense/{split}_pos.json', indent=4, orient='records')

In [ ]:
"""sem-eval DATA"""

In [8]:
import pandas as pd

split = 'dev'
data_sem_eval = pd.read_csv(f'./sem-eval/{split}.csv')

data = []
for index, row in data_sem_eval.iterrows():
    statement_f = row['Incorrect Statement']
    explanation_f = row['Right Reason1']
    data.append({'id': index,
                 'statement': statement_f,
                 'explanation': explanation_f,
                 'label': 0})

data = pd.DataFrame(data)
data.to_json(f'./com2sense/{split}_neg.json', indent=4, orient='records')

"\ndata = []\nfor index, row in data_sem_eval.iterrows():\n    statement_f = row['Incorrect Statement']\n    explanation_f = row['Right Reason1']\n    data.append({'id': index,\n                 'statement': statement_f,\n                 'explanation': explanation_f,\n                 'label': 0})\n\ndata = pd.DataFrame(data)\ndata.to_json(f'./com2sense/{split}_neg.json', indent=4, orient='records')\n"

In [6]:
"""TRUNCATE 2 DATASETS"""
import pandas as pd
split = 'train'
data_pos = pd.read_json(f'com2sense/{split}_pos.json')
data_neg = pd.read_json(f'com2sense/{split}_neg.json')

if len(data_pos) < len(data_neg):
    data_neg = data_neg[0: len(data_pos)]
else:
    data_pos = data_pos[0: len(data_neg)]

data = pd.concat([data_neg, data_pos])
data.to_json(f'./com2sense/{split}.json', indent=4, orient='records')
